# Project #2: Facial Verification

## Importing Necessary Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

!pip install facenet_pytorch
from facenet_pytorch import MTCNN
from PIL import Image
import os
from tqdm import tqdm

## Preprocessing my Images

In [ ]:
#defining file paths for each set used in data
pos_training = "/content/drive/MyDrive/Project2_Data/positives/training"
pos_test = "/content/drive/MyDrive/Project2_Data/positives/test"
pos_validation = "/content/drive/MyDrive/Project2_Data/positives/validation"

neg_training = "/content/drive/MyDrive/Project2_Data/negatives/training"
neg_test = "/content/drive/MyDrive/Project2_Data/negatives/test"
neg_validation = "/content/drive/MyDrive/Project2_Data/negatives/validation"

folders = {
    "positives_training": pos_training,
    "positives_test": pos_test,
    "positives_validation": pos_validation,
    "negatives_training": neg_training,
    "negatives_test": neg_test,
    "negatives_validation": neg_validation,
}


In [ ]:
#output folder to store results
base_output = "/content/drive/MyDrive/Project2_Data/aligned"
os.makedirs(base_output, exist_ok=True)
for name in folders.keys():
    os.makedirs(os.path.join(base_output, name), exist_ok=True)

#defining MTCNN
mtcnn = MTCNN(image_size=160, margin=10, keep_all=False, device='cpu')

def align_faces(input_folder, output_folder):
    print(f"\nProcessing: {input_folder}")
    for filename in tqdm(os.listdir(input_folder)):
        if filename.startswith('.'):
            continue  # skip hidden files
        filepath = os.path.join(input_folder, filename)
        if not os.path.isfile(filepath):
            continue
        try:
            img = Image.open(filepath).convert("RGB")
            face = mtcnn(img)
            if face is not None:
                # Convert back to image
                aligned_img = Image.fromarray((face.permute(1, 2, 0).numpy() * 255).astype('uint8'))
                save_path = os.path.join(output_folder, filename)
                aligned_img.save(save_path)
            else:
                print(f"No face detected in {filename}")
        except Exception as e:
            print(f"Error processing {filename}: {e}")

#performs function on each folder
for name, path in folders.items():
    out_path = os.path.join(base_output, name)
    align_faces(path, out_path)


Processing: /content/drive/MyDrive/Project2_Data/positives/training


100%|██████████| 32/32 [06:21<00:00, 11.91s/it]



Processing: /content/drive/MyDrive/Project2_Data/positives/test


100%|██████████| 13/13 [02:27<00:00, 11.37s/it]



Processing: /content/drive/MyDrive/Project2_Data/positives/validation


...

100%|██████████| 25/25 [03:16<00:00,  7.88s/it]



Processing: /content/drive/MyDrive/Project2_Data/negatives/test


100%|██████████| 12/12 [01:32<00:00,  7.70s/it]



Processing: /content/drive/MyDrive/Project2_Data/negatives/validation


100%|██████████| 11/11 [01:15<00:00,  6.84s/it]


# Running Models

## Embeddings + Threshold

In [31]:
from facenet_pytorch import InceptionResnetV1
import torch
from torchvision import transforms

# Load pretrained FaceNet model
resnet = InceptionResnetV1(pretrained='vggface2').eval()

# Transform for model input
transform = transforms.Compose([
    transforms.Resize((160, 160)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

#function to create enbedding for each image, or numerical representation of each image
def get_embedding(image_path):
    img = Image.open(image_path).convert('RGB')
    img_tensor = transform(img).unsqueeze(0)  # shape (1,3,160,160)
    with torch.no_grad():
        embedding = resnet(img_tensor)
    return embedding.squeeze().numpy()  # shape (512,)


In [32]:
#new aligned file paths
p_aligned_training = "/content/drive/MyDrive/Project2_Data/aligned/positives_training"
p_aligned_test = "/content/drive/MyDrive/Project2_Data/aligned/positives_test"
p_aligned_validation = "/content/drive/MyDrive/Project2_Data/aligned/positives_validation"

n_aligned_training = "/content/drive/MyDrive/Project2_Data/aligned/negatives_training"
n_aligned_test = "/content/drive/MyDrive/Project2_Data/aligned/negatives_test"
n_aligned_validation = "/content/drive/MyDrive/Project2_Data/aligned/negatives_validation"

In [34]:
# Helper: extract embeddings from all images in a folder
def get_embeddings_from_folder(folder_path, label):
    embeddings = []
    labels = []

    for filename in tqdm(os.listdir(folder_path), desc=f"Processing {os.path.basename(folder_path)}"):
        filepath = os.path.join(folder_path, filename)
        if os.path.isfile(filepath) and not filename.startswith('.'):
            try:
                emb = get_embedding(filepath)
                embeddings.append(emb)
                labels.append(label)
            except Exception as e:
                print(f"Error processing {filepath}: {e}")
    return embeddings, labels

# Process all splits
X_train_p, y_train_p = get_embeddings_from_folder(p_aligned_training, 1)
X_train_n, y_train_n = get_embeddings_from_folder(n_aligned_training, 0)

X_val_p, y_val_p = get_embeddings_from_folder(p_aligned_validation, 1)
X_val_n, y_val_n = get_embeddings_from_folder(n_aligned_validation, 0)

# Combine each split
X_train = np.array(X_train_p + X_train_n)
y_train = np.array(y_train_p + y_train_n)

X_val = np.array(X_val_p + X_val_n)
y_val = np.array(y_val_p + y_val_n)

Processing negatives_validation: 100%|██████████| 10/10 [00:01<00:00,  9.36it/s]


In [35]:
print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)

(55, 512) (55,)
(21, 512) (21,)


In [36]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import classification_report, roc_curve

#this reference embedding will be used to give new images reference point to average embedding of positive images
anchor_embedding = np.mean(X_train[y_train == 1], axis=0)

#calculating the cosine similarities of each image in the validation set
similarities = cosine_similarity(X_val, anchor_embedding.reshape(1, -1)).flatten()

# Validation similarities
val_sim = cosine_similarity(X_val, anchor_embedding.reshape(1, -1)).flatten()

# Compute ROC to find best threshold
fpr, tpr, thresholds = roc_curve(y_val, val_sim)
best_idx = np.argmax(tpr - fpr)
best_threshold = thresholds[best_idx]

print(f"Best threshold = {best_threshold:.3f}")

Best threshold = 0.768


In [25]:
y_pred = (similarities >= best_threshold).astype(int)

print(classification_report(y_val, y_pred, digits=3, target_names=['Negative', 'Positive']))

              precision    recall  f1-score   support

    Negative      0.636     0.700     0.667        10
    Positive      0.700     0.636     0.667        11

    accuracy                          0.667        21
   macro avg      0.668     0.668     0.667        21
weighted avg      0.670     0.667     0.667        21



## CNN Classifier

In [8]:
import tensorflow as tf
import numpy as np
import os
from tensorflow.keras.preprocessing.image import load_img, img_to_array

#specifying image size to be used on CNN
IMG_SIZE = (160, 160)

#function to load each image from the respective files
def load_images_from_folder(folder, label):
    images, labels = [], []
    for filename in os.listdir(folder):
        if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
            path = os.path.join(folder, filename)
            img = load_img(path, target_size=IMG_SIZE)
            img = img_to_array(img) / 255.0
            images.append(img)
            labels.append(label)
    return np.array(images), np.array(labels)

X_pos_train, y_pos_train = load_images_from_folder(p_aligned_training, 1)
X_neg_train, y_neg_train = load_images_from_folder(n_aligned_training, 0)

# Combine and shuffle
X_train = np.concatenate([X_pos_train, X_neg_train])
y_train = np.concatenate([y_pos_train, y_neg_train])

# Repeat for validation/test
X_pos_val, y_pos_val = load_images_from_folder(p_aligned_validation, 1)
X_neg_val, y_neg_val = load_images_from_folder(n_aligned_validation, 0)

X_val = np.concatenate([X_pos_val, X_neg_val])
y_val = np.concatenate([y_pos_val, y_neg_val])

X_pos_test, y_pos_test = load_images_from_folder(p_aligned_test, 1)
X_neg_test, y_neg_test = load_images_from_folder(n_aligned_test, 0)

X_test = np.concatenate([X_pos_test, X_neg_test])
y_test = np.concatenate([y_pos_test, y_neg_test])

In [18]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#uses augmentation to increase diversity of training set
train_datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    brightness_range=[0.8, 1.2],
    zoom_range=0.1,
    horizontal_flip=True
)

train_generator = train_datagen.flow(X_train, y_train, batch_size=16)
val_generator = train_datagen.flow(X_val, y_val, batch_size=16)

from tensorflow import keras
from tensorflow.keras import layers

#creating layers of CNN with convolution, pooling, and dense layers
inputs = keras.Input(shape=(160, 160, 3))
x = layers.Conv2D(32, (3,3), activation='relu')(inputs)
x = layers.MaxPooling2D(2)(x)
x = layers.Conv2D(64, (3,3), activation='relu')(x)
x = layers.MaxPooling2D(2)(x)
x = layers.Conv2D(128, (3,3), activation='relu')(x)
x = layers.Flatten()(x)
x = layers.Dense(128, activation='relu')(x)
outputs = layers.Dense(1, activation='sigmoid')(x)

model = keras.Model(inputs=inputs, outputs=outputs)

#defining model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

#fitting model
model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=20
)

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 8s 1s/step - accuracy: 0.5108 - loss: 0.7140 - val_accuracy: 0.4762 - val_loss: 0.6613
Epoch 2/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 7s 2s/step - accuracy: 0.6867 - loss: 0.6780 - val_accuracy: 0.4286 - val_loss: 0.6941
Epoch 3/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 0.6472 - loss: 0.6784 - val_accuracy: 0.5238 - val_loss: 0.6706
Epoch 4/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 0.5855 - loss: 0.6755 - val_accuracy: 0.5238 - val_loss: 0.6846
Epoch 5/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 7s 2s/step - accuracy: 0.5747 - loss: 0.6634 - val_accuracy: 0.5238 - val_loss: 0.7095
Epoch 6/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 0.5609 - loss: 0.6785 - val_accuracy: 0.6190 - val_loss: 0.6032
Epoch 7/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - accuracy: 0.6617 - loss: 0.6628 - val_accuracy: 0.4762 - val_loss: 0.6973
Epoch 8/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 9s 1s/step - accuracy: 0.6422 - loss: 0.6557 - val_accuracy: 0.5238 - val_loss: 0.6751
Epoch 9/20
4/4 ━

In [19]:
y_pred_proba = model.predict(val_generator)
y_pred = (y_pred_proba > 0.5).astype(int).flatten()

print(classification_report(y_val, y_pred, target_names=['Negative', 'Positive']))

2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 162ms/step
              precision    recall  f1-score   support

    Negative       0.83      0.50      0.62        10
    Positive       0.67      0.91      0.77        11

    accuracy                           0.71        21
   macro avg       0.75      0.70      0.70        21
weighted avg       0.75      0.71      0.70        21



A key consideration in model selection for my selected use case was the recall performance and false positive rate, as I would prefer my model be more secure in detecting negatives rather than select positive correctly everytime. Given this, I will be selecting the Pretrained Embeddings + Baseline model. Please refer to report for further discussion of this decision.

# Running Pretrained Embeddings + Baseline on Test Set

In [37]:
#need to reprocess splits
X_train_p, y_train_p = get_embeddings_from_folder(p_aligned_training, 1)
X_train_n, y_train_n = get_embeddings_from_folder(n_aligned_training, 0)

X_test_p, y_test_p = get_embeddings_from_folder(p_aligned_test, 1)
X_test_n, y_test_n = get_embeddings_from_folder(n_aligned_test, 0)

#recombining splits
X_train = np.array(X_train_p + X_train_n)
y_train = np.array(y_train_p + y_train_n)

X_test = np.array(X_test_p + X_test_n)
y_test = np.array(y_test_p + y_test_n)

#training on same data that was used on validation set
anchor_embedding = np.mean(X_train[y_train == 1], axis=0)

#calculating similarities on test set
similarities = cosine_similarity(X_test, anchor_embedding.reshape(1, -1)).flatten()

Processing negatives_test: 100%|██████████| 11/11 [00:01<00:00,  9.29it/s]


In [38]:
y_pred = (similarities >= best_threshold).astype(int)

print(classification_report(y_test, y_pred, digits=3, target_names=['Negative', 'Positive']))

              precision    recall  f1-score   support

    Negative      0.600     0.818     0.692        11
    Positive      0.750     0.500     0.600        12

    accuracy                          0.652        23
   macro avg      0.675     0.659     0.646        23
weighted avg      0.678     0.652     0.644        23

